# *CoastSat*: example at Eva Island, Australia

The underlying software is described in details in:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 30+ years at their site of interest.
There are three main steps:
1. Retrieval of the satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution
3. Intersection of the shorelines with cross-shore transects

This toolkit extends this application to islands by using the satellite-derived shorelines to calculate 2D planform characteristics (area and orientation). Application of this is described in: 
* Cuttler MVW, Vos K, Branson P, Hansen JE, O'Leary M, Browne NK, Lowe RJ (2020) Reef island response to climate-driven variations in water level and wave climate (under review)


## Initial settings

Refer to the **Installation** section of the README for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. If that step has been completed correctly, the following packages should be imported without any problem.

In [1]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import SDS_download, SDS_preprocess
import SDS_island_tools, SDS_island_shorelines, SDS_island_transects


## 1. Retrieval of the images from GEE

Define the region of interest (`polygon`), the date range (`dates`) and the satellite missions (`sat_list`) from which you wish to retrieve the satellite images. The images will be cropped on the Google Earth Engine server and only the region of interest will be downloaded as a .tif file. The files will stored in the directory defined in `filepath`. 

Make sure the area of your ROI is smaller than 100 km2 (if larger split it into smaller ROIs).

The function `SDS_download.check_images_available(inputs)` will print the number of images available for your inputs. The Landsat images are divided in Tier 1 and Tier 2, only Tier 1 images can be used for time-series analysis. 

In [2]:
# region of interest (longitude, latitude)
polygon = SDS_island_tools.coords_from_kml(os.path.join(os.getcwd(), 'example','EVA.kml'))
           
# date range
dates = ['2019-01-01', '2019-02-01']
# satellite missions
sat_list = ['S2']
# name of the site
sitename = 'EVA'
# directory where the data will be stored
filepath = os.path.join(os.getcwd(), 'data')
#island file - info about island slope and center coordinates
island_file = os.path.join(os.getcwd(), 'example',sitename + '_info.csv')
# put all the inputs into a dictionnary
inputs = {'polygon': polygon, 'dates': dates, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath, 'island_file': island_file}

# before downloading the images, check how many images are available for your inputs
SDS_download.check_images_available(inputs);

Images available between 2019-01-01 and 2019-02-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 24 images
  Total: 24 images


The function `SDS_download.retrieve_images(inputs)` retrives the satellite images from Google Earth Engine.

By default, only Landsat Tier 1 and Sentinel-2 Level-1C are downloaded. 

In case you need to access Tier 2 images for qualitative analysis, you need to set `inputs['include_T2'] = True` before calling `retrieve_images`.

In [3]:
# inputs['include_T2'] = True
metadata = SDS_download.retrieve_images(inputs)

Images available between 2019-01-01 and 2019-02-01:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  S2: 24 images
  Total: 24 images

S2: 12 images
100%
0 Sentinel-2 images were merged (overlapping or duplicate)


**If you have already retrieved the images**, just load the metadata file by only running the section below

In [ ]:
metadata = SDS_download.get_metadata(inputs) 

## 2. Shoreline extraction

This section maps the position of the shoreline on the satellite images. The user can define the cloud threhold (`cloud_thresh`) and select the spatial reference system in which to output the coordinates of the mapped shorelines (`output_epsg`). See http://spatialreference.org/ to find the EPSG number corresponding to your local coordinate system. Make sure that your are using cartesian coordinates and not spherical coordinates (lat,lon) like WGS84. 

To quality control each shoreline detection and manually validate the mapped shorelines, the user has the option to set the parameter `check_detection` to **True**. To save a figure for each mapped shoreline set `save_figure` to **True**. 

The other parameters are for advanced users only and are described in the README.

In [4]:
settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': 28350,       # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection_sand_poly': True,    # if True, shows each shoreline detection to the user for validation
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # add the inputs defined previously
    'inputs': inputs,
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    'zref': 0                   #reference height datum for tidal correction 
}

#read additional settings for island info - adds:
settings = SDS_island_tools.read_island_info(island_file,settings)

### [OPTIONAL] Save .jpg of the satellite images 
Saves .jpg files of the preprocessed satellite images (cloud masking + pansharpening/down-sampling) under *./data/sitename/jpeg_files\preprocessed*

In [5]:
SDS_preprocess.save_jpg(metadata, settings)

Satellite images saved as .jpg in C:\Users\00084142\Dropbox\Research\Active_Projects\CUTTLER_GitHub\CoastSat.islands\data\EVA\jpg_files\preprocessed


### [OPTIONAL] Digitize a reference shoreline
Creates a reference shoreline which helps to identify outliers and false detections. The reference shoreline is manually digitised by the user on one of the images. The parameter `max_dist_ref` defines the maximum distance from the reference shoreline (in metres) at which a valid detected shoreline can be. If you think that the default value of 100 m will not capture the full shoreline variability of your site, increase this value to an appropriate distance.

In [6]:
%matplotlib qt
settings['reference_shoreline'] = SDS_preprocess.get_reference_sl(metadata, settings)
settings['max_dist_ref'] = 100 # max distance (in meters) allowed from the reference shoreline

Reference shoreline already exists and was loaded


### Batch shoreline detection
Extracts the 2D shorelines from the images in the spatial reference system specified by the user in `'output_epsg'`. The mapped shorelines are saved into `output.pkl` (under *./data/sitename*) and `output.geojson` (to be used in a GIS software).

If you see that the sand pixels on the images are not being identified, change the parameter `sand_color` from `default` to `dark` or `bright` depending on the color of your beach. 

In [8]:
%matplotlib qt
output = SDS_island_shorelines.extract_shorelines(metadata, settings)

Mapping shorelines:
2019-01-03-02-43-48_S2_EVA_10m.tif
S2:   8%2019-01-05-02-33-57_S2_EVA_10m.tif
S2:   16%2019-01-08-02-43-53_S2_EVA_10m.tif
S2:   25%2019-01-10-02-33-54_S2_EVA_10m.tif
S2:   33%2019-01-13-02-43-50_S2_EVA_10m.tif
S2:   41%2019-01-15-02-33-58_S2_EVA_10m.tif
S2:   50%2019-01-18-02-43-54_S2_EVA_10m.tif
S2:   58%2019-01-20-02-33-55_S2_EVA_10m.tif
S2:   66%2019-01-23-02-43-51_S2_EVA_10m.tif
S2:   75%2019-01-25-02-33-59_S2_EVA_10m.tif
S2:   83%2019-01-28-02-43-54_S2_EVA_10m.tif
S2:   91%2019-01-30-02-33-56_S2_EVA_10m.tif
S2:   100%


Simple plot of the mapped shorelines. The coordinates are stored in the output dictionnary together with the exact dates in UTC time, the georeferencing accuracy and the cloud cover.

In [ ]:
fig = plt.figure()
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
plt.legend()
mng = plt.get_current_fig_manager()                                         
mng.window.showMaximized()    
fig.set_size_inches([15.76,  8.52])

In [17]:
fig = plt.figure()
plt.plot(output['dates'],output['sand_area'],'b-x')
plt.grid('on')
plt.xlabel('Date')
plt.ylabel('Sub-aerial sand area (m^2)')
mng = plt.get_current_fig_manager()                                         
mng.window.showMaximized()    
fig.set_size_inches([15.76,  8.52])

## 3. Shoreline analysis

In this section we show how to compute time-series of cross-shore distance along user-defined shore-normal transects.

**If you have already mapped the shorelines**, just load the output file (`output.pkl`) by running the section below

In [18]:
filepath = os.path.join(inputs['filepath'], sitename)
with open(os.path.join(filepath, sitename + '_output' + '.pkl'), 'rb') as f:
    output = pickle.load(f) 

There are 3 options to define the coordinates of the shore-normal transects:

**Option 1**: the user can interactively draw the shore-normal transects along the beach by calling:

In [ ]:
%matplotlib qt
transects = SDS_island_transects.draw_transects(output, settings)

**Option 2**: the user can load the transect coordinates (make sure the spatial reference system is the same as defined previously by the parameter *output_epsg*) from a .geojson file by calling:

In [ ]:
geojson_file = os.path.join(os.getcwd(), 'examples', 'NARRA_transects.geojson')
transects = SDS_island_tools.transects_from_geojson(geojson_file)

**Option 3**: manually provide the coordinates of the transects as shown in the example below:

In [ ]:
transects = dict([])
transects['Transect 1'] = np.array([[342836, 6269215], [343315, 6269071]])
transects['Transect 2'] = np.array([[342482, 6268466], [342958, 6268310]])
transects['Transect 3'] = np.array([[342185, 6267650], [342685, 6267641]])

**Option 4**: calculate transects emanating from single origin point (e.g. for islands):

In [22]:
ang_start = 0 #start heading (degrees)
ang_end = 360 #end heading (degrees)
ang_step = 10 #degree step for calculating transects 
settings['transect_length'] = 400 
settings['heading'] = np.array(list(range(ang_start,ang_end,ang_step)))
transects = SDS_island_transects.calc_island_transects(settings)          


   

Now, intersect the transects with the 2D shorelines to obtain time-series of cross-shore distance.

The time-series of shoreline change for each transect are saved in a .csv file in the data folder (all dates are in UTC time). 

In [25]:
# defines the along-shore distance over which to consider shoreline points to compute the median intersection (robust to outliers)
settings['along_dist'] = 25  
 
cross_distance = SDS_island_transects.compute_intersection(output, transects, settings) 

Calculating intersections: 100%Time-series of the shoreline change along the transects saved as:
C:\Users\00084142\Dropbox\Research\Active_Projects\CUTTLER_GitHub\CoastSat.islands\data\EVA\transect_time_series.csv


Tide correct the polygons

In [26]:
#process tide data
#input tide data is in local time (Australian West Coast, UTC +8 hrs), but code below converts to UTC
tide_file = os.path.join(os.getcwd(),'example','EvaTide_2019.txt')
tide, output_corrected = SDS_island_tools.process_tide_data(tide_file, output)    

cross_distance_corrected = SDS_island_tools.tide_correct(cross_distance,tide,settings['zref'],settings['beach_slope'])
       
#Calculate tidally corrected sand_polygon     
output_corrected = SDS_island_tools.tide_correct_sand_polygon(cross_distance_corrected, output_corrected, settings)
    

Calculating tides: 100%

TypeError: 'int' object is not subscriptable